Rough Notes: 1-Combined Approach
2- Ensemble of models
3-Inter Intra Class Approach

ToDo:<br/>  1- Add TfIdf scores for prompt relevancy in dataset
            2- Change Scaling to a better type
            3- Log Model 

Reverse Scaling: X = (X_norm * (max_val - min_val)) + min_val

**Training error NaN loss:**
<br/>1. Increased Dropout,
<br/>2. Decreased LR, Batchsize
<br/>3. Removed NaN values
<br/>4. clipvalue



# **Importing Packages**

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
import nltk
import gensim.downloader as api
from nltk.tokenize import word_tokenize, sent_tokenize

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding,Masking,LSTM, GRU, Conv1D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SimpleRNN

# **Dataset Preparation**

In [35]:
from google.colab import drive
drive.mount('/content/drive')
df_train01 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet01_Features.csv", index_col =0)
df_train02 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet02_Features.csv", index_col =0)
df_train03 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet03_Features.csv", index_col =0)
df_train04 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet04_Features.csv", index_col =0)
df_train05 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet05_Features.csv", index_col =0)
df_train06 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet06_Features.csv", index_col =0)
df_train07 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet07_Features.csv", index_col =0)
df_train08 = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet08_Features.csv", index_col =0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
dfs = []
for i in range(1, 9):
    df = pd.read_csv(f'/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet0{i}_Features.csv')  # assuming the dataframes are stored as CSV files
    scores_2d = [[score] for score in df['Total Score']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    df['Total Score'] = scaler.fit_transform(scores_2d)   #Changed from z-score to Minmax, z-score was giving negative range
    df['Essay Set'] = i
    dfs.append(df[['ID', 'Essay Set', 'Total Score', 'Preprocessed_Essay']])  # extract only the ID and Score columns and append to the list

# concatenate the dataframes using pd.concat() and the loc accessor
df_train = pd.concat(dfs, axis=0).reset_index(drop=True)
df_train['Grade'] = df_train['Essay Set'].apply(lambda x: 8 if x == 1 or x == 5 else (7 if x == 7 else 10))
df_train['Essay Type'] = df_train['Essay Set'].apply(lambda x: 'source dependent responses' if x in range(3,7) else 'persuasive / narrative  / expository')


In [37]:
nltk.download('punkt') 
#df_train['tokenized_text'] = df_train['Preprocessed_Essay'].apply(lambda x: [word_tokenize(sent.lower()) for sent in sent_tokenize(x)])
# df_train['tokenized_text'] = df_train['tokenized_text'].apply(lambda x: ' '.join(x))
df_train['tokenized_text'] = df_train['Preprocessed_Essay'].apply(lambda x: word_tokenize(x.lower()))
df_train.sample(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ID,Essay Set,Total Score,Preprocessed_Essay,Grade,Essay Type,tokenized_text
4999,7398,3,0.333333,in the towns that the cyclist arrived at were ...,10,source dependent responses,"[in, the, towns, that, the, cyclist, arrived, ..."
6319,9876,4,0.333333,the author concludes in the story that saeng f...,10,source dependent responses,"[the, author, concludes, in, the, story, that,..."
4606,7004,3,0.666667,the features of the setting affected the cycli...,10,source dependent responses,"[the, features, of, the, setting, affected, th..."
4393,6790,3,0.666667,in the story ‘do not exceed posted speed limit...,10,source dependent responses,"[in, the, story, ‘, do, not, exceed, posted, s..."
3404,4599,2,0.400000,should materials such as books music movies...,10,persuasive / narrative / expository,"[should, materials, such, as, books, music, mo..."


In [38]:
def para_to_sent(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    return sentences

# Apply function to 'Essay' column of the dataframe
df_train['Essay Sentences'] = df_train['Preprocessed_Essay'].apply(para_to_sent)

In [39]:
df_train.shape

(12978, 8)

In [40]:
df_train['Essay Sentences']

0        [dear local newspaper  i think effects compute...
1        [dear i believe that using computers will bene...
2        [dear  more and more people use computers  but...
3        [dear local newspaper  i have found that many ...
4        [dear i know having computers has a positive e...
                               ...                        
12973    [ in most stories mothers and daughters are ei...
12974    [ i never understood the meaning laughter is t...
12975    [when you laugh  is out of habit  or is cause ...
12976    [                               trippin  on fe...
12977    [ many people believe that laughter can improv...
Name: Essay Sentences, Length: 12978, dtype: object

In [41]:
df_train.to_csv("Dataset.csv")

# **Data Cleaning**
Note: Change scaling type to something more inclined with non-linear data type and, remove NaNs from essay set04

In [42]:
# Finding the number of missing values in dataset
missing = df_train.isnull().sum()
print(missing)
# Finding percentage of missing value
total_cells = np.product(df_train.shape)
percent = (missing.sum() / total_cells)*100
percent  

ID                    0
Essay Set             0
Total Score           1
Preprocessed_Essay    0
Grade                 0
Essay Type            0
tokenized_text        0
Essay Sentences       0
dtype: int64


0.0009631684388965943

In [43]:
df_train = df_train.dropna()

In [44]:
df_train.shape

(12977, 8)

In [45]:
# Finding the number of missing values in dataset
missing = df_train.isnull().sum()
print(missing)
# Finding percentage of missing value
total_cells = np.product(df_train.shape)
percent = (missing.sum() / total_cells)*100
percent  

ID                    0
Essay Set             0
Total Score           0
Preprocessed_Essay    0
Grade                 0
Essay Type            0
tokenized_text        0
Essay Sentences       0
dtype: int64


0.0

# **Word Embedding of Preprocessed Essay**



In [46]:
glove_model = api.load("glove-wiki-gigaword-100")

In [47]:
def get_glove_embeddings(text_data):
    # Tokenize the text data
    tokenized_data = [sentence.split() for sentence in text_data]

    # Obtain the GloVe embeddings
    embeddings = np.zeros((len(text_data), 100))

    for i, sentence in enumerate(tokenized_data):
        print(i  , sentence)
        break
        for word in sentence:
            if word in glove_model.key_to_index:
                embeddings[i] += glove_model[word]
        embeddings[i] /= len(sentence)
    return embeddings

# Obtain the GloVe embeddings for the text data
glove_embeddings = get_glove_embeddings(df['Preprocessed_Essay'])

0 ['a', 'long', 'time', 'ago', 'when', 'i', 'was', 'in', 'third', 'grade', 'i', 'had', 'a', 'friend', 'who', 's', 'mom', 'was', 'in', 'a', 'bad', 'mood', 'she', 'never', 'laughed', 'and', 'she', 'never', 'smiled', 'every', 'time', 'i', 'saw', 'her', 'i', 'would', 'smile', 'at', 'her', 'and', 'all', 'she', 'would', 'do', 'was', 'frown', 'and', 'keep', 'walking', 'at', 'first', 'i', 'didn', 't', 'know', 'she', 'was', 'a', 'grouch', 'i', 'just', 'thought', 'she', 'didn', 't', 'like', 'me', 'or', 'something', 'when', 'told', 'me', 'his', 'mom', 'was', 'a', 'grouch', 'i', 'started', 'to', 'laugh', 'and', 'laugh', 'he', 'asked', 'me', 'what', 'was', 'so', 'funny', 'i', 'told', 'him', 'that', 'i', 'thought', 'his', 'mom', 'didn', 't', 'like', 'me', 'or', 'something', 'because', 'every', 'time', 'i', 'see', 'his', 'mom', 'i', 'would', 'smile', 'at', 'her', 'and', 'all', 'she', 'will', 'do', 'is', 'frown', 'and', 'walk', 'away', 'that', 'made', 'my', 'friend', 'laugh', 'we', 'were', 'cracking',

In [48]:
glove_embeddings.shape

(723, 100)

In [49]:
# Define a function to obtain the GloVe embeddings
def get_glove_embeddings(text_data):
    # Flatten the list of lists
    flattened_data = [sentence for sublist in text_data for sentence in sublist]
    
    # Join the sentences with a space character
    joined_data = [' '.join(sentence) for sentence in flattened_data]

    # Obtain the GloVe embeddings
    embeddings = np.zeros((len(joined_data), 100))  

    for i, sentence in enumerate(joined_data):
        for word in sentence.split():
            if word in glove_model.key_to_index:
                embeddings[i] += glove_model[word]
        embeddings[i] /= len(sentence.split())
    return embeddings

# Obtain the GloVe embeddings for the text data
glove_embeddings = get_glove_embeddings(df['Preprocessed_Essay'])


<ipython-input-49-3133353da120>:16: RuntimeWarning: invalid value encountered in true_divide
  embeddings[i] /= len(sentence.split())


In [50]:
glove_embeddings.shape

(2163263, 100)

# **Categorical Features Encoding**

In [51]:
# 0 for persuasive / narrative / expository and 1 for source dependent responses
df_train['Essay Type'] = df_train['Essay Type'].str.replace(
    'persuasive / narrative  / expository', '0', case=True, regex=False)

df_train['Essay Type'] = df_train['Essay Type'].str.replace(
    'source dependent responses', '1', case=True, regex=False)

# Ordinal Encoding
oe = OrdinalEncoder(categories=[['7', '8', '10']])   # encoded as 0, 1 and 2
df_train['Grade'] = oe.fit_transform(df_train[['Grade']]).astype(int)

In [52]:
df_train.sample(5)

,ID,Essay Set,Total Score,Preprocessed_Essay,Grade,Essay Type,tokenized_text,Essay Sentences
11438,18676,7,1.000000,i have been a patient man in my life an examp...,0,0,"[i, have, been, a, patient, man, in, my, life,...",[i have been a patient man in my life an exam...
11167,18373,7,0.545455,one day i was looking at a catalog in a store ...,0,0,"[one, day, i, was, looking, at, a, catalog, in...",[one day i was looking at a catalog in a store...
5100,7500,3,0.333333,how the features of the setting affected the c...,2,1,"[how, the, features, of, the, setting, affecte...",[how the features of the setting affected the ...
7006,10567,4,0.000000,i think the author concluded the story like t...,2,1,"[i, think, the, author, concluded, the, story,...",[i think the author concluded the story like ...
2383,3578,2,0.600000,today in society there are many things people ...,2,0,"[today, in, society, there, are, many, things,...",[today in society there are many things people...


In [53]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12977 entries, 0 to 12977
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  12977 non-null  int64  
 1   Essay Set           12977 non-null  int64  
 2   Total Score         12977 non-null  float64
 3   Preprocessed_Essay  12977 non-null  object 
 4   Grade               12977 non-null  int64  
 5   Essay Type          12977 non-null  object 
 6   tokenized_text      12977 non-null  object 
 7   Essay Sentences     12977 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 912.4+ KB


# **Model Implementation**

In [54]:
vocab_size = 10000
embedding_dim = 100  #from 16
max_length = 100
trunc_type='post' # remove sentences if they reach the max_lenght
padding_type='post' #padding after sentences
oov_tok = "<OOV>"   #placeholder for new words in padding
training_size = 20000
DROPOUT_RATE = 0.5

## **Encoded Input Text**

In [55]:
tk = Tokenizer(num_words=vocab_size)
tokenized_text = df_train['tokenized_text'].values.tolist()
flatten_text = [word for sent in tokenized_text for word in sent]
joined_text = ' '.join(flatten_text)
tk.fit_on_texts([joined_text])

encoded_essay_train = tk.texts_to_sequences(tokenized_text)
padded_essay_train = pad_sequences(encoded_essay_train, maxlen=max_length, padding=padding_type)   #Input Text

In [56]:
print(padded_essay_train.dtype)
padded_essay_train.shape

int32


(12977, 100)

## **Input Features**

In [57]:
feature_cols = ['Essay Set', 'Essay Type','Grade']   #add tf-idf
features_df = df_train[feature_cols]
# Convert features DataFrame to NumPy array
features_train = features_df.values

print(features_train.dtype)
print(features_train.shape)
features_train = features_train.astype('float32')
print(features_train.dtype)

object
(12977, 3)
float32


## **Target Variable**

In [58]:
target_train = df_train['Total Score']
print(target_train.dtype)
print(target_train.shape)

float64
(12977,)


## **Vanilla LSTM**

In [59]:
################################################################## Try One ##########################################################

# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.LSTM(units=64),  #default activation is relu
#     tf.keras.layers.Dense(units=1, activation='linear')
# ])
# model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

In [60]:
################################################################# Try Two #############################################################

# model = Sequential()
# model.add(Embedding(input_dim=embedding_matrix.shape[0],
#                     output_dim=EMBEDDING_DIM,
#                     weights=[embedding_matrix],
#                     input_length=MAX_SEQ_LEN,
#                     trainable=False))
# model.add(LSTM(units=64, activation='sigmoid'))
# model.add(Dropout(rate=DROPOUT_RATE))
# model.add(Dense(units=1, activation='linear'))


In [61]:
#IF NEEDED
def cohen_kappa_with_quadratic_weights(y_true, y_pred):
    y_true_classes = tf.round(y_true)
    y_pred_classes = tf.round(y_pred)
    kappa = cohen_kappa_score(y_true_classes, y_pred_classes, weights='quadratic')
    return kappa

In [62]:
# Note: Reduced batch size from 2048 to 128.
def build_vanilla_lstm(MAX_SEQ_LEN = 100, EMBEDDING_DIM = 100, DROPOUT_RATE = 0.5,EPOCHS = 100, BATCH_SIZE = 128 ,VALIDATION_SPLIT = 0.2,embedding_matrix=None):   
# NOTE: Modify the function so it can accept parameters for later use in gridsearch cv  and bayesian and then make one function for bidriectional, add callback line that bahram told and plot the reslts like in tensorflow.ipynb
 
  np.random.seed(42)
  tf.random.set_seed(42)
  NUM_FEATURES = len(features_train[0])
  # Load pre-trained GloVe embeddings
  #embedding_matrix = {} 
  # Define input layers
  input_text = Input(shape=(MAX_SEQ_LEN,), dtype='int32')   # Padded Essay Tokens
  input_features = Input(shape=(NUM_FEATURES,), dtype='float32') # Input Features

  # Define embedding layer
  embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                              output_dim=EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQ_LEN,
                              trainable=False)
  
  # Define LSTM layer
  lstm_layer = LSTM(units=64, activation='tanh')
  # Connect input layers to embedding and LSTM layers
  text_embed = embedding_layer(input_text)
  text_lstm = lstm_layer(text_embed)
  # Concatenate LSTM output with input features
  concat_layer = concatenate([text_lstm, input_features])
  # Define dropout layer
  dropout_layer = Dropout(rate=DROPOUT_RATE)
  # Connect concat layer to dropout layer
  dropout_output = dropout_layer(concat_layer)
  # Define output layer
  output_layer = Dense(units=1, activation='linear')   #try it with sigmoid, relu, and tanh
  # Connect dropout output to output layer
  output = output_layer(dropout_output)

  # Define model with two inputs and one output
  model = tf.keras.Model(inputs=[input_text, input_features], outputs=output)

  # Compile model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00000001,clipvalue=1.0),   #try sgd, bgd,RMSprop
                loss='mse',
                metrics=['mae'])
  return model


### **Training Model**

In [ ]:
EPOCHS = 100 
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2

lstm_model = build_vanilla_lstm(MAX_SEQ_LEN = 100, EMBEDDING_DIM = 100, DROPOUT_RATE = 0.5,EPOCHS = 100, BATCH_SIZE = 128 ,VALIDATION_SPLIT = 0.2, embedding_matrix=glove_embeddings)
lstm_model.fit([padded_essay_train, features_train], target_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_split=VALIDATION_SPLIT)

Epoch 1/100
82/82 [==============================] - 25s 275ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
82/82 [==============================] - 18s 222ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
82/82 [==============================] - 18s 217ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
82/82 [==============================] - 18s 218ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
82/82 [==============================] - 17s 205ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
82/82 [==============================] - 18s 220ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
82/82 [==============================] - 17s 209ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
82/82 [==============================] - 18s 219ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
82/82 [=====

In [ ]:
lstm_model.summary()   #Confusion: MaxLenght=100 should be dropped? Since words in a sentences/Para can exceed the limit

### **Results Visualization**

In [ ]:
history = lstm_model.history.history

import matplotlib.pyplot as plt

# Plot validation loss
plt.plot(history['val_loss'], label='val_loss')
plt.plot(history['loss'], label='train_loss')
plt.legend(loc="best")
plt.show()

# Plot evaluation metrics
plt.plot(history['mae'], label='train_mae')
plt.plot(history['val_mae'], label='val_mae')
plt.legend(loc="best")
plt.show()

### **Testing Model**

In [ ]:
test_loss, test_mae = model.evaluate([padded_docs_test, features_test], target_test)

# Make predictions on test data
predictions = model.predict([padded_docs_test, features_test])

# Compute Cohen's kappa score
kappa = cohen_kappa_score(np.round(predictions), target_test, weights='quadratic')

print('Test Loss:', test_loss)
print('Test MAE:', test_mae)
print('Cohen\'s Kappa Score:', kappa)

## **GridSearchCV** (Incomplete)

## **BiDirectional LSTM** (Incomplete)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input, concatenate, Bidirectional

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define model parameters
MAX_SEQ_LEN = 100
EMBEDDING_DIM = 100
DROPOUT_RATE = 0.5
NUM_FEATURES = len(features_train[0])
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2

# Load pre-trained GloVe embeddings
embedding_matrix = {} # Load GloVe embeddings here

# Define input layers
input_text = Input(shape=(MAX_SEQ_LEN,), dtype='int32')
input_features = Input(shape=(NUM_FEATURES,), dtype='float32')

# Define embedding layer
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LEN,
                            trainable=False)

# Define bidirectional LSTM layer
lstm_layer = Bidirectional(LSTM(units=64, activation='sigmoid'))

# Connect input layers to embedding and LSTM layers
text_embed = embedding_layer(input_text)
text_lstm = lstm_layer(text_embed)

# Concatenate LSTM output with input features
concat_layer = concatenate([text_lstm, input_features])

# Define dropout layer
dropout_layer = Dropout(rate=DROPOUT_RATE)

# Connect concat layer to dropout layer
dropout_output = dropout_layer(concat_layer)

# Define output layer
output_layer = Dense(units=1, activation='linear')

# Connect dropout output to output layer
output = output_layer(dropout_output)

# Define model with two inputs and one output
model = tf.keras.Model(inputs=[input_text, input_features], outputs=output)

# Compile model
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

# Train model
model.fit([padded_docs_train, features_train], target_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_split=VALIDATION_SPLIT)
